# Lab 3: EDA and Data Prep

Exploratory Data Analysis (EDA) is the initial phase of data analysis, focusing on understanding data characteristics, patterns, and relationships through visualizations and summaries. Data Preparation in machine learning involves preprocessing raw data to ensure its suitability for model training, including tasks like handling missing values, encoding variables, scaling features, and splitting data for training and testing. Both EDA and Data Preparation are crucial stages for extracting insights and preparing data for machine learning models.

### Exercise overview

In this exercise, you'll explore the dataset through EDA, uncovering its patterns and relationships. You'll also perform tasks like data preprocessing, handling missing values, and feature engineering to make the data ready for machine learning training.

### Helpful links
- [Data Wrangler in Microsoft Fabric](https://aka.ms/fabric/datawrangler)

## Pre-Requisite

For this Exercise, we expect that you have completed and ran **Lab 2: Access your data through semantic link**. 


## Step 1: Setup your notebook

### Select Lakehouse (FC_Workshop)
First, add the Lakehouse you created from the prior lab exercise.

<br>

![image-alt-text](https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/add-lakehouse.png)

## Step 2: Load the data as a Spark DataFrame

Load the data that you have created and saved in the lakehouse from the previous notebook.

In [ ]:
df = spark.sql("SELECT * FROM FC_Workshop.churnfromsemanticlink")
display(df)

Convert the spark DataFrame to pandas DataFrame for easier processing and visualization.

In [ ]:
df = df.toPandas()

### Dataset

The dataset contains churn status of 10,000 customers along with 13 attributes that include last credit score, geographical location (Germany, France, Spain), gender (male, female), age, tenure (years of being bank's customer), account balance, estimated salary, number of products that a customer has purchased through the bank, credit card status (whether a customer has a credit card or not), and active member status (whether an active bank's customer or not).

The dataset also includes columns such as account ID, customer ID, and customer surname that should have no impact on customer's decision to leave the bank. The event that defines the customer's churn is the closing of the customer's bank account, therefore, the column `Exited` in the dataset refers to customer's abandonment. Since you don't have much context about these attributes, you'll proceed without having background information about the dataset. Your aim is to understand how these attributes contribute to the `Exited` status.

Out of the 10,000 customers, only 2037 customers (around 20%) have left the bank. Therefore, given the class imbalance ratio, it is recommended to generate synthetic data.

- churn.csv

|"CustomerId"|"Surname"|"Last Credit Score"|"Geography"|"Gender"|"Age"|"Tenure"|"Balance"|"Number Of Products"|"Has Credit Card"|"Is Active"|"EstimatedSalary"|"Exited"|"AccountId"|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|15634602|Hargrave|619|France|Female|42|2|0.00|1|1|1|101348.88|True|4602|
|15647311|Hill|608|Spain|Female|41|1|83807.86|1|0|1|112542.58|False|8401|

## Step 3: Perform Exploratory Data Analysis

Explore the raw data with `display`, do some basic statistics and show chart views. You first need to import required libraries for data visualization such as `seaborn` which is a Python data visualization library to provide a high-level interface for building visuals on dataframes and arrays. Learn more about [`seaborn`](https://seaborn.pydata.org/). 

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid", palette="tab10", rc = {'figure.figsize':(9,6)})
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rc, rcParams
import numpy as np
import pandas as pd
import itertools

### Display raw data

In [ ]:
display(df, summary=True)

### Use Data Wrangler to perform initial data cleansing

To explore and transform any pandas Dataframes in your notebook, launch Data Wrangler directly from the notebook.

>[!NOTE]
>Data Wrangler can not be opened while the notebook kernel is busy. The cell execution must complete prior to launching Data Wrangler.

1. Under the notebook ribbon Data tab, select Launch Data Wrangler. You'll see a list of activated pandas DataFrames available for editing.
<!-- 1. Under the notebook ribbon Home tab, launch the Data Wrangler in order to see a list of activated pandas DataFrames available for editing. -->
2. Select the DataFrame you wish to open in Data Wrangler. Since this notebook only contains one DataFrame at this point, select `df`.

<br>

<img src="https://sdkstorerta.blob.core.windows.net/churnblob/select_datawrangler.png"  width="40%" height="10%" title="Screenshot shows where to access the Data Wrangler.">

<br>

Data Wrangler launches and generates a descriptive overview of your data. The table in the middle shows each data column. 

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/DataWrangler_2.png"  width="40%" height="10%" title="Screenshot shows where to access the Data Wrangler.">


The Summary panel next to the table shows information about the DataFrame. When you select a column in the table, the summary updates with information about the selected column. In some instances, the data displayed and summarized will be a truncated view of your DataFrame. When this happens, you'll see warning image in the summary pane. Hover over this warning to view text explaining the situation.

Each operation you do can be applied in a matter of clicks, updating the data display in real time and generating code that you can save back to your notebook as a reusable function.

The rest of this section walks you through the steps to perform data cleaning with Data Wrangler.

##### Drop duplicate rows

On the left panel is a list of operations (such as `Find and replace`, `Format`, `Formulas`, `Numeric`) you can perform on the dataset.

1. Expand `Find and replace` and select `Drop duplicate rows`.

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/DataWrangler_3.png"  width="40%" height="10%" title="Screenshot shows where to access the Data Wrangler.">

2. A panel appears for you to select the list of columns you want to compare to define a duplicate row. Select `CustomerId`, and `AccountId`.

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/DataWrangler_4.png"  width="40%" height="10%" title="Screenshot shows where to access the Data Wrangler.">


In the middle panel is a preview of the results of this operation. Under the preview is the code to perform the operation. In this instance, the data appears to be unchanged. But since you're looking at a truncated view, it's a good idea to still apply the operation.

3. Select `Apply` (either at the side or at the bottom) to go to the next step.

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/DataWrangler_5.png"  width="40%" height="10%" title="Screenshot shows where to access the Data Wrangler.">


##### Drop rows with missing data

Similar to above, now use Data Wrangler to drop rows with missing data across all columns.

1. Select `Drop missing values` from `Find and replace`.

2. Choose `Select all` from the `Target columns`.

3. Select `Apply` to go on to the next step.

##### Drop columns

Finally, use Data Wrangler to drop columns that you don't need.

1. Expand `Schema` and select `Drop columns`.

2. Select `CustomerId`, `Surname`, and `AccountId`. These columns appear in red in the preview, to show they're changed by the code (in this case, dropped.)

3.Select `Apply` to go on to the next step.

##### Add code to notebook

Each time you select `Apply`, a new step is created in the `Cleaning steps` panel on the bottom left. At the bottom of the panel, select `Preview code for all steps` to view a combination of all the separate steps.

Select `Add code to notebook` at the top left to close Data Wrangler and add the code automatically. The `Add code to notebook` wraps the code in a function, then calls the function.

>[!NOTE]
>The code generated by Data Wrangler won't be applied until you manually run the new cell in the notebook.

<img src="https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/DataWrangler_6.png"  width="40%" height="10%" title="Screenshot shows where to access the Data Wrangler.">


## Exercise: Follow the steps above and add the code generated by Data Wrangler


In [ ]:
# TODO: Add the code generated by Data Wrangler from the above data cleaning steps

### 

### Unlocking Insights with EDA and Visualization

##### Determine attributes

Use this code to determine categorical, numerical, and target attributes.

In [ ]:
# Determine the dependent (target) attribute
dependent_variable_name = "Exited"
print(dependent_variable_name)
# Determine the categorical attributes
categorical_variables = [col for col in df_clean.columns if col in "O"
                        or df_clean[col].nunique() <=5
                        and col not in "Exited"]
print(categorical_variables)
# Determine the numerical attributes
numeric_variables = [col for col in df_clean.columns if df_clean[col].dtype != "object"
                        and df_clean[col].nunique() >5]
print(numeric_variables)


##### The five-number summary 

Show the five-number summary (the minimum score, first quartile, median, third quartile, the maximum score) for the numerical attributes, using box plots.

In [ ]:
df_num_cols = df_clean[numeric_variables]
sns.set(font_scale = 0.7) 
fig, axes = plt.subplots(nrows = 2, ncols = 3, gridspec_kw =  dict(hspace=0.3), figsize = (17,8))
fig.tight_layout()
for ax,col in zip(axes.flatten(), df_num_cols.columns):
    sns.boxplot(x = df_num_cols[col], color='green', ax = ax)
# fig.suptitle('visualize and compare the distribution and central tendency of numerical attributes', color = 'k', fontsize = 12)
fig.delaxes(axes[1,2])


##### Distribution of exited and non-exited customers 

Show the distribution of exited versus non-exited customers across the categorical attributes.

In [ ]:
attr_list = ['Geography', 'Gender', 'Has Credit Card', 'Is Active', 'Number Of Products', 'Tenure']
fig, axarr = plt.subplots(2, 3, figsize=(15, 4))
for ind, item in enumerate (attr_list):
    sns.countplot(x = item, hue = 'Exited', data = df_clean, ax = axarr[ind%2][ind//2])
fig.subplots_adjust(hspace=0.7)

##### Distribution of numerical attributes

Show the the frequency distribution of numerical attributes using histogram.


In [ ]:
columns = df_num_cols.columns[: len(df_num_cols.columns)]
fig = plt.figure()
fig.set_size_inches(18, 8)
length = len(columns)
for i,j in itertools.zip_longest(columns, range(length)):
    plt.subplot((length // 2), 3, j+1)
    plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
    df_num_cols[i].hist(bins = 20, edgecolor = 'black')
    plt.title(i)
# fig = fig.suptitle('distribution of numerical attributes', color = 'r' ,fontsize = 14)
plt.show()

##### Summary of observations from the exploratory data analysis

- Most of the customers are from France comparing to Spain and Germany, while Spain has the lower churn rate comparing to France and Germany.
- Most of the customers have credit cards.
- There are customers whose age and credit score are above 60 and below 400, respectively, but they can't be considered as outliers.
- Very few customers have more than two of the bank's products.
- Customers who aren't active have a higher churn rate.
- Gender and tenure years don't seem to have an impact on customer's decision to close the bank account.

## Step 4: Perform feature engineering 

The following feature engineering generates new attributes based on current attributes.

In [ ]:
df_clean["NewTenure"] = df_clean["Tenure"]/df_clean["Age"]
df_clean["NewCreditsScore"] = pd.qcut(df_clean['Last Credit Score'], 6, labels = [1, 2, 3, 4, 5, 6])
df_clean["NewAgeScore"] = pd.qcut(df_clean['Age'], 8, labels = [1, 2, 3, 4, 5, 6, 7, 8])
df_clean["NewBalanceScore"] = pd.qcut(df_clean['Balance'].rank(method="first"), 5, labels = [1, 2, 3, 4, 5])
df_clean["NewEstSalaryScore"] = pd.qcut(df_clean['EstimatedSalary'], 10, labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
df_clean['Exited'] = df['Exited'].astype(int)

## Step 5: Use Data Wrangler to perform one-hot encoding

Data Wrangler can also be used to perform one-hot encoding. To do so, re-open Data Wrangler. This time, select the `df_clean` data.

1. Expand `Formulas` and select `One-hot encode`.
2. A panel appears for you to select the list the column you want to perform one-hot encoding on. Select `Geography`.

You should be able to see in the preview that column `Geography` is dropped, while three new columns `Geography_Germany`, `Geography_France`, and `Geography_Spain` are added as a result of one-hot encoding.

1. Expand `Schema` and select `Change column type`.
2. A panel appears for you to select the list of columns you want to change their type as target columns. Select `Geography_Germany`, `Geography_France`, and `Geography_Spain` as the newly added columns to the DataFarme.
3. Select the `New type` as `float64`.
3. Select `Apply` (either at the side or at the bottom) to go to the next step.

You could copy the generated code, close Data Wrangler to return to the notebook, then paste into a new cell. Or, select `Add code to notebook` at the top left to close Data Wrangler and add the code automatically.

<br>

<img style="float: left;" src="https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/OneHotEncoding1.png"  width="45%" height="20%" title="Screenshot shows one-hot encoding in the Data Wrangler"> 
<img style="float: left;" src="https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/OneHotEncoding2.png"  width="45%" height="20%" title="Screenshot shows selection of columns in the Data Wrangler.">



In [ ]:
# Code generated by Data Wrangler for pandas DataFrame

import pandas as pd

def clean_data(df_clean):
    # One-hot encode column: 'Geography'
    insert_loc = df_clean.columns.get_loc('Geography')
    df_clean = pd.concat([df_clean.iloc[:,:insert_loc], pd.get_dummies(df_clean.loc[:, ['Geography']]), df_clean.iloc[:,insert_loc+1:]], axis=1)
    # Change column type to float64 for columns: 'Geography_France', 'Geography_Germany', 'Geography_Spain'
    df_clean = df_clean.astype({'Geography_France': 'float64', 'Geography_Germany': 'float64', 'Geography_Spain': 'float64'})
    return df_clean

df_clean_1 = clean_data(df_clean.copy())
df_clean_1.head()

You can repeat the above steps to perform one-hot encoding on other columns, e.g., `Gender`, `Had Credit Card`, and `Is Active`. In the following, the code generated by the Data Wrangler for each of the columns is added to the notebook.

In [ ]:
# Code generated by Data Wrangler for pandas DataFrame

import pandas as pd

def clean_data(df_clean_1):
    # One-hot encode column: 'Gender'
    insert_loc = df_clean_1.columns.get_loc('Gender')
    df_clean_1 = pd.concat([df_clean_1.iloc[:,:insert_loc], pd.get_dummies(df_clean_1.loc[:, ['Gender']]), df_clean_1.iloc[:,insert_loc+1:]], axis=1)
    # Change column type to float64 for columns: 'Gender_Female', 'Gender_Male'
    df_clean_1 = df_clean_1.astype({'Gender_Female': 'float64', 'Gender_Male': 'float64'})
    return df_clean_1

df_clean_2 = clean_data(df_clean_1.copy())
df_clean_2.head()

In [ ]:
# Code generated by Data Wrangler for pandas DataFrame

import pandas as pd

def clean_data(df_clean_2):
    # One-hot encode column: 'Has Credit Card'
    insert_loc = df_clean_2.columns.get_loc('Has Credit Card')
    df_clean_2 = pd.concat([df_clean_2.iloc[:,:insert_loc], pd.get_dummies(df_clean_2.loc[:, ['Has Credit Card']]), df_clean_2.iloc[:,insert_loc+1:]], axis=1)
    # Change column type to float64 for columns: 'Has Credit Card_No', 'Has Credit Card_Yes'
    df_clean_2 = df_clean_2.astype({'Has Credit Card_No': 'float64', 'Has Credit Card_Yes': 'float64'})
    return df_clean_2

df_clean_3 = clean_data(df_clean_2.copy())
df_clean_3.head()

In [ ]:
# Code generated by Data Wrangler for pandas DataFrame

import pandas as pd

def clean_data(df_clean_3):
    # One-hot encode column: 'Is Active'
    insert_loc = df_clean_3.columns.get_loc('Is Active')
    df_clean_3 = pd.concat([df_clean_3.iloc[:,:insert_loc], pd.get_dummies(df_clean_3.loc[:, ['Is Active']]), df_clean_3.iloc[:,insert_loc+1:]], axis=1)
    # Change column type to float64 for columns: 'Is Active_No', 'Is Active_Yes'
    df_clean_3 = df_clean_3.astype({'Is Active_No': 'float64', 'Is Active_Yes': 'float64'})
    return df_clean_3

df_clean_4 = clean_data(df_clean_3.copy())
df_clean_4.head()

## Step 6: Rename Columns

In the provided code snippet, a dictionary named `column_mapping` is defined to map existing column names in a DataFrame, which contain white spaces and underscores, to new, more concise column names without white spaces. This mapping is then used to rename the columns of a DataFrame named `df_clean_4` using its `rename` method. The result of this operation is stored in a new DataFrame called `df_clean_renamed`, where the column names are updated according to the mappings specified.

In [ ]:
# Rename columns to avoid white space
# Create a dictionary to map old column names to new column names
column_mapping = {
    "Has Credit Card_No": "HasCreditCard",
    "Has Credit Card_Yes": "HasCreditCard_Yes",
    "Is Active_No": "IsActive_No",
    "Is Active_Yes": "IsActive_Yes",
    "Number Of Products": "NumberOfProducts",
    "Last Credit Score": "LastCreditScore",
}
# Rename columns using the rename method
df_clean_renamed = df_clean_4.rename(columns=column_mapping)

## Step 7: Save final data to lakehouse
Finally, we will save the cleaned data to the lakehouse to be used for next step. 

In [ ]:
table_name = "churn_data_clean"

# Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(df_clean_renamed) 
sparkDF.write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")